In [ ]:
import pandas as pd
import ipywidgets as widgets
import csv

In [ ]:
ElementDF = pd.read_csv('RuleElements.csv')
SelectorList = pd.read_csv('selectors.csv').columns.to_list()
OptionList = pd.read_csv('options.csv').columns.to_list()
CommonPredicates = ['agile','finesse','rage','unarmed','fear','mental','poison','disease','fear','sweep']
DamageTypes = ['','slashing','piercing','bludgeoning','precision','fire','cold','electricity','acid','poison','sonic','force','mental','lawful','chaotic','good','evil','positive','negative']

In [ ]:
def MakeRuleElement(Key,Options):
    ####################################################################
    # This takes in a key and options for that key and builds a rule element.
    #
    # key -- A string giving the key for the rule element.
    # options -- An array of strings containing the options for the rule element
    ####################################################################
    
    RuleElement = '{"key\":\"' + Key + '"'
    
    for Option in Options:
        RuleElement += ',' + Option
    
    RuleElement += '}'
    
    return print(RuleElement)

def GetPropertyList(row):
    ####################################################################
    # This returns a list of the applicable rule elements.
    #
    # To be used with df.apply(), this is a utility function and just
    # helps to store values.  It does not need to be run except to add 
    # new properties.
    ####################################################################
    
    row = row[PropertyList]
    
    passed = row[row == 'y'].index.tolist()
    return(passed)

def PreparePredicates(All, Any, Not):
    ####################################################################
    # This puts the predacate lists together into a single entity
    #
    # All -- List containing predicates which must all be satisfied
    # Any -- List containing predicates where one must be satified.
    # Not -- !All
    ####################################################################
    
    Predicate = str()
    
    if len(All + Any + Not) > 0:
        Predicate += '\"predicate\":{'
    
    if len(All) > 0:
        Predicate += '\"all\":[\"'
        i = 1
        for each in All:
            if i == len(All):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'
        
        if len(Any + Not) > 0:
            Predicate += ','
            
    if len(Any) > 0:
        Predicate += '\"any\":[\"'
        i = 1
        for each in Any:
            if i == len(Any):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'
        
        if len(Not) > 0:
            Predicate += ','
            
    if len(Not) > 0:
        Predicate += '\"not\":[\"'
        i = 1
        for each in Not:
            if i == len(Not):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'

    if len(All + Any + Not) > 0:
        Predicate += '}'
        
    return Predicate


def PrepareOptions(Selector, AdvancedSelector, Value, Type, DamageType, DieSize, DiceNumber, Predicate, Label, Name, Override):
    ####################################################################
    # This brings each option together and puts them into a single string.
    #
    # All inputs are strings with the exception of value, which is an integer.
    ####################################################################
    
    Options = []
    
    if len(Selector) > 0:
        Options += ['"selector":"' + AdvancedSelector + Selector + '"']
    
    if len(DamageType) > 0:
        if DamageType == 'precision':
            Options += ['"damageCategory":"' + DamageType + '"']
        else:
            Options += ['"damageType":"' + DamageType + '"']

    if len(DieSize) > 0:
        if Override:
            Options += ['"override":{"dieSize":"' + DieSize + '"}']
        else:
            Options += ['"dieSize":"' + DieSize + '"']
        
    if DiceNumber > 0:
        Options += ['"diceNumber":' + str(DiceNumber)]
    
    if Value > 0:
        Options += ['"value":' + str(Value)]
        
    if len(Type) > 0:
        Options += ['"type":"' + Type + '"']
        
    if len(Predicate) >0:
        Options += [Predicate]
    
    if Label:
        Options += ['"label":"YOUR TEXT HERE"']
    
    if Name:
        Options += ['"name":"YOUR TEXT HERE"']
        
    return Options

In [ ]:
PropertyList = SelectorList
ElementDF['ValidSelectors'] = ElementDF.apply(GetPropertyList,axis = 1)

PropertyList = OptionList
ElementDF['ValidOptions'] = ElementDF.apply(GetPropertyList,axis = 1)

In [ ]:
Key = widgets.Dropdown(
    options=ElementDF.FriendlyName.tolist(),
    value=ElementDF.FriendlyName.tolist()[0],
    description='Key:',
    disabled=False,
)

display(Key)

In [ ]:
Selector = widgets.Dropdown(
    options=ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0],
    value=ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0][0],
    description='selector',
    disabled=False
)

AdvancedSelector = widgets.Dropdown(
    options=[('no advanced selector',''),('Only this item','{item|_id}-'),('swords','sword-weapon-group-'),('bows','bow-weapon-group-'),('knives','knife-weapon-group-'),('spears','spear-weapon-group-'),('axes','axe-weapon-group-'),('hammers','hammer-weapon-group-'),('picks','pick-weapon-group-'),('flails','flail-weapon-group-'),('brawling','brawling-weapon-group-'),('polearms','polearm-weapon-group-'),('sling','sling-weapon-group-'),('bombs','bomb-weapon-group-'),('shields','shield-weapon-group-'),('darts','dart-weapon-group-'),('clubs','clubs-weapon-group-')],
    value='',
    description='Advanced Selector',
    disabled=False
)

display(Selector)

display(AdvancedSelector)

In [ ]:
if 'damage' in Selector.value:
    DamageType = widgets.Dropdown(
    options=DamageTypes,
    value='',
    description='Damage type',
    disabled=False )
    
    display(DamageType)
else:
    DamageType = str()

if 'value' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
    Value = widgets.IntText(
    value=1,
    description='Value:',
    disabled=False)
    
    display(Value)
else:
    Value = 0

if 'diceNumber' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
    DiceNumber = widgets.IntText(
    value=1,
    description='Number of dice:',
    disabled=False )
    
    display(DiceNumber)
else:
    DiceNumber = 0
    
if 'dieSize' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
    DieSize = widgets.Dropdown(
    options=['','d4','d6','d8','d10','d12'],
    value='d4',
    description='Die size:',
    disabled=False )
    
    display(DieSize)
    
    Override = widgets.Checkbox(
    value=False,
    description='Override base damage?',
    disabled=False,
    indent=False )
    
    display(Override)
else:
    DieSize = str()
    Override = False

if 'type' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
    Type = widgets.Dropdown(
    options=['','circumstance','item','status'],
    value='',
    description='Type',
    disabled=False )
    
    display(Type)
else: Type = str()

if 'predicate' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
    All = widgets.SelectMultiple(
    options=CommonPredicates,
    value=[],
    description='Predicate:All',
    disabled=False )
    Any = widgets.SelectMultiple(
    options=CommonPredicates,
    value=[],
    description='Any',
    disabled=False )
    Not = widgets.SelectMultiple(
    options=CommonPredicates,
    value=[],
    description='Not',
    disabled=False )

    display(All)
    display(Any)
    display(Not)
else:
    All = []
    Any = []
    Not = []
    
Label = widgets.Checkbox(
    value=False,
    description='Add a label?',
    disabled=False,
    indent=False
)

Name = widgets.Checkbox(
    value=False,
    description='Add a Name?',
    disabled=False,
    indent=False
)

display(Label)
display(Name)

In [ ]:
Key = ElementDF.Key[ElementDF.FriendlyName == Key.value].values[0]

try:
    Selector = Selector.value
except:
    pass

try:
    AdvancedSelector = AdvancedSelector.value
except:
    pass

try:
    Value = Value.value
except:
    pass

try:
    Type = Type.value
except:
    pass

try:
    DamageType = DamageType.value
except:
    pass

try:
    DieSize = DieSize.value
except:
    pass

try:
    DiceNumber = DiceNumber.value
except:
    pass

try:
    All = All.value
except:
    pass

try:
    Any = Any.value
except:
    pass

try:
    Not = Not.value
except:
    pass

try:
    Override = Override.value
except:
    pass

Name = Name.value
Label = Label.value

Predicate = PreparePredicates(All, Any, Not)
Options = PrepareOptions(Selector, AdvancedSelector, Value, Type, DamageType, DieSize, DiceNumber, Predicate, Label, Name, Override)

MakeRuleElement(Key,Options)